# Census ACS

Retrieves data from the Census Bureau's American Community Survey 5-year series API for zctas, pumas, and tracts. A specific list of census variables is passed into the script, which are retrieved from the four ACS profile tables. Variables must be retrieved in chunks because only 50 can be passed to the API at a time, and each url varies by geography and retreives them in different combinations. After some processing output is written to a SQLite database. An option to create a metadata table appears at the bottom, but should only be run once for a given extract (acs1 and acs2) and not for each individual geography.

https://www.census.gov/data/developers/data-sets/acs-5year.html

## Variables

In [1]:
import os, requests, json, sqlite3, random, pandas as pd, numpy as np
from IPython.display import clear_output

In [2]:
keyfile='census_key.txt'

#API variables - UPDATE THE YEAR AND GEO
year='2018'
geo='tract' # 'zip code tabulation area' or 'public use microdata area' or 'tract'
state='44'
dsource='acs'
dsource2='acs5'
dname='profile'

#Variables to read in from spreadsheet - UPDATE WORKSHEET
worksheet='acs1' # 'acs1' or 'acs2'
geoexcelsheet={'zip code tabulation area':'zctas', 'public use microdata area':'pumas', 'tract':'tracts'}
geotype=geoexcelsheet.get(geo)

#SQL output
tabname='{}_{}{}'.format(geotype,year,worksheet)
dbname=os.path.join('outputs','testdb.sqlite')

#Dump files for api data storage
jsonpath=os.path.join('outputs', tabname+'_retrieved_data.json')

## Variable Lists
Get full list of variables from the API, read in our retrieval list, and compare the varianle IDs and names to make sure nothing is missing and that nothing has changed since the last iteration. *Don't move on to the next block until both lists match.* Lastly, read in list of geographies.

In [3]:
datadict={}
dps=['DP02','DP03','DP04','DP05']
for p in dps:
    vars_url = f'https://api.census.gov/data/{year}/{dsource}/{dsource2}/{dname}/groups/{p}.json'
    response=requests.get(vars_url)
    var_data=response.json()
    datadict.update(var_data['variables'])
random.sample(list(datadict.items()), 2)

[('DP02_0076PMA',
  {'label': 'Percent Annotation of Margin of Error!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!65 years and over',
   'concept': 'SELECTED SOCIAL CHARACTERISTICS IN THE UNITED STATES',
   'predicateType': 'string',
   'group': 'DP02',
   'limit': 0,
   'predicateOnly': True}),
 ('DP05_0070PEA',
  {'label': 'Percent Annotation of Estimate!!HISPANIC OR LATINO AND RACE!!Total population',
   'concept': 'ACS DEMOGRAPHIC AND HOUSING ESTIMATES',
   'predicateType': 'string',
   'group': 'DP05',
   'limit': 0,
   'predicateOnly': True})]

In [4]:
dfexcel = pd.read_excel(os.path.join('inputs','acs_variables.xlsx'),sheet_name=worksheet)
dfexcel.head()

,db_var,census_var,census_label,dtype
0,HSHD01_E,DP02_0001E,Estimate!!HOUSEHOLDS BY TYPE!!Total households,int
1,HSHD01_M,DP02_0001M,Margin of Error!!HOUSEHOLDS BY TYPE!!Total hou...,int
2,HSHD01_PC,DP02_0001PE,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total ho...,int
3,HSHD01_PM,DP02_0001PM,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!T...,int
4,HSHD02_E,DP02_0002E,Estimate!!HOUSEHOLDS BY TYPE!!Total households...,int


In [5]:
dfvars = pd.DataFrame.from_dict(datadict,columns=['label'],orient='index')
dfvars_selected=dfvars.loc[dfvars.index.isin(dfexcel['census_var'])]
dfvars_count=len(dfvars_selected)
dfexcel_count=len(dfexcel['census_var'])

if dfvars_count==dfexcel_count:
    print('There are an equal number of variables in both lists:', dfvars_count)
else:
    print('There is a mismatch in the number of variables; the api has,', dfvars_count, 
          'while the original list has',dfexcel_count,'. Missing:')
    nomatch=dfexcel[~dfexcel['census_var'].isin(dfvars_selected.index)]
    print(nomatch)

There are an equal number of variables in both lists: 248


In [6]:
mismatch=dfexcel[~dfexcel['census_label'].isin(dfvars_selected['label'])]

if len (mismatch) ==0:
    print('All labels match')
else:
    compare=pd.merge(mismatch,dfvars_selected, left_on='census_var', right_on=dfvars_selected.index)
    misfile=os.path.join('outputs','{}_mismatch.csv'.format(worksheet))
    compare.to_csv(misfile, columns=['db_var', 'census_var', 'census_label', 'label'],sep=',', index=False,
    header=['db_var','census_var' ,'oldlabel','newlabel'])
    print('* Mismatch file printed to outputs folder * \n')
    print('These labels do not match:')
    print(compare[['census_var','census_label','label']])

All labels match


In [7]:
# Geographic indetifiers: zctas to retrieve, pumas to filter by, and counties containing tracts to retrieve
excelgeo = pd.read_excel(os.path.join('inputs','acs_variables.xlsx'),sheet_name=geotype, dtype=object)
geoids = excelgeo['GEO'].tolist()
print('Number of geographic indetifiers:',len(geoids))

Number of geographic indetifiers: 5


## Retrieve Data
Given the large number of variables in the ACS and limits of the API, variables must be passed to the url in separate blocks or chunks. The first chunk that's captured is written to an empty datalist; the header row and then one row for each geography. Each subsequent chunk is iterated through by row, so each row is appended to the correct row in datalist. In all cases, the last values, identifiers automatically returned with each API call, are not appended.

In [8]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [9]:
reqvars=list(chunks(dfvars_selected.index.tolist(),46))
reqvars[0].insert(0,'NAME')
reqvars[0].insert(0,'GEO_ID')
print('Number of chunks:',len(reqvars))

Number of chunks: 6


In [10]:
with open(keyfile) as key:
    api_key=key.read().strip()

base_url = f'https://api.census.gov/data/{year}/{dsource}/{dsource2}/{dname}'
base_url

'https://api.census.gov/data/2018/acs/acs5/profile'

In [11]:
#Function for retrieving data; running this block loads it into memory
#Different geographies have different urls, 
#and a different number of identifiers tacked on to the end of each request

def getdata():
    dlist=[]
    for i, v in enumerate(reqvars):
        batchcols=','.join(v)
        if geotype=='zctas':
            data_url = f'{base_url}?get={batchcols}&for={geo}:{g}&key={api_key}'
            dropvar=-1
        elif geotype=='pumas':
            data_url = f'{base_url}?get={batchcols}&for={geo}:*&in=state:{state}&key={api_key}'
            dropvar=-2
        elif geotype=='tracts':
            data_url = f'{base_url}?get={batchcols}&for={geo}:*&in=state:{state}&in=county:{county}&key={api_key}'
            dropvar=-3
        else:
            print('Appropriate geography not specified in variables block')
            break  
        response=requests.get(data_url)
        if response.status_code==200:
            clear_output(wait=True)
            data=response.json()
            for i2, v2 in enumerate(data):
                if i == 0:
                    dlist.append(v2[:dropvar])
                else:
                    for item in v2[:dropvar]:
                        dlist[i2].append(item)
        else:
            print('***Problem with retrieval***, response code',response.status_code)
    return dlist

#### ***THIS BLOCK IS A REQUESTS BLOCK!***
*NOTE: ZCTA retrieval takes a long time - 15 mins for 215 ZCTAs*

In [12]:
#If this block was run successfully for a given table and geography don't rerun - next block pulls from saved json
datalist=[]
if geotype=='zctas':
    for g in geoids:
        georecord=getdata()
        print('Retrieved data for',g)
        if len(datalist)==0:
            datalist.append(georecord[0])
            datalist.append(georecord[1])
        else:
            datalist.append(georecord[1])
elif geotype=='pumas':
    datalist=getdata()
elif geotype=='tracts':
    for county in geoids:
        georecord=getdata()
        print('Retrieved data for',county)
        if len(datalist)==0:
            for tract in georecord:
                datalist.append(tract)
        else:
            for tract in georecord[1:]:
                datalist.append(tract)
    
dlrows=len(datalist)
dlitems=sum(len(x) for x in datalist)
dlbyrow=dlitems / dlrows
print('Retrieved', dlrows, 'records and', dlitems,'data points with', dlbyrow, 'points for each record...')
        
with open(jsonpath, 'w') as f:
    json.dump(datalist, f)
print('Done - Data dumped to json file')

Retrieved data for 009
Retrieved 245 records and 61250 data points with 250.0 points for each record...
Done - Data dumped to json file


## Process Data
Replace footnotes with nulls, create a new GEOID2 column, replace census variable names with database variable names.

In [36]:
with open(jsonpath, 'r') as f:
    jsondata=json.load(f)
alldata = pd.DataFrame(jsondata[1:],columns=jsondata[0],dtype=object).rename(columns={
    'GEO_ID':'GEOIDLONG','NAME':'GEOLABEL'}).set_index('GEOIDLONG')
alldata.info()
# Index and column entries should be 1 row and 1 column less than previous count (excludes header row and index column) 

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, 1400000US44001030100 to 1400000US44009051306
Columns: 249 entries, GEOLABEL to DP03_0063PM
dtypes: object(249)
memory usage: 476.6+ KB


In [37]:
alldata.head(3)

,GEOLABEL,DP02_0001E,DP02_0001M,DP02_0001PE,DP02_0001PM,DP02_0002E,DP02_0002M,DP02_0002PE,DP02_0002PM,DP02_0003E,...,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM
GEOIDLONG,,,,,,,,,,,,,,,,,,,,,
1400000US44001030100,"Census Tract 301, Bristol County, Rhode Island",1828,81,1828,-888888888,1258,92,68.8,4.8,669,...,18.4,4.9,96875,13302,-888888888,-888888888,121685,12991,-888888888,-888888888
1400000US44001030200,"Census Tract 302, Bristol County, Rhode Island",1207,83,1207,-888888888,862,94,71.4,6.9,342,...,14.7,5.7,106875,11018,-888888888,-888888888,127146,16401,-888888888,-888888888
1400000US44001030300,"Census Tract 303, Bristol County, Rhode Island",1654,77,1654,-888888888,1316,79,79.6,4.6,651,...,23.6,4.8,131757,8480,-888888888,-888888888,160177,15896,-888888888,-888888888


In [15]:
#This is a lousy solution, come up with something better in the future
footnotes=['-999999999','-999999999.0', '-999999999.00',
           '-888888888','-888888888.0', '-888888888.00',
           '-666666666','-666666666.0', '-666666666.00',
           '-555555555','-555555555.0', '-555555555.00',
           '-333333333','-333333333.0', '-333333333.00',
           '-222222222','-222222222.0', '-222222222.00']
alldata.replace(footnotes,np.nan,inplace=True)

In [35]:
idxgeoid2={'zctas':-5, 'pumas':-7,'tracts':-11}
alldata.insert(loc=0, column='GEOIDSHORT',value=alldata.index.str[idxgeoid2.get(geotype):])
alldata.head(3)

,GEOIDSHORT,GEOID2,GEOLABEL,DP02_0001E,DP02_0001M,DP02_0001PE,DP02_0001PM,DP02_0002E,DP02_0002M,DP02_0002PE,...,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM
GEOID,,,,,,,,,,,,,,,,,,,,,
1400000US44001030100,44001030100,44001030100,"Census Tract 301, Bristol County, Rhode Island",1828,81,1828,NaN,1258,92,68.8,...,18.4,4.9,96875,13302,NaN,NaN,121685,12991,NaN,NaN
1400000US44001030200,44001030200,44001030200,"Census Tract 302, Bristol County, Rhode Island",1207,83,1207,NaN,862,94,71.4,...,14.7,5.7,106875,11018,NaN,NaN,127146,16401,NaN,NaN
1400000US44001030300,44001030300,44001030300,"Census Tract 303, Bristol County, Rhode Island",1654,77,1654,NaN,1316,79,79.6,...,23.6,4.8,131757,8480,NaN,NaN,160177,15896,NaN,NaN


In [17]:
# For PUMAS filter all the geotype for the state by local areas
if geotype == 'pumas':
    acsdata=alldata.loc[alldata.GEOID2.isin(geoids)].copy().astype(object).sort_index()
else:
    acsdata=alldata.copy().astype(object).sort_index()
acsdata.shape

(244, 250)

In [20]:
#Dictionary of column names from the census and the nyc geodatabase
cv_to_db=dict(zip(dfexcel.census_var, dfexcel.db_var))
random.sample(list(cv_to_db.items()), 5)

[('DP02_0062E', 'EDU05_E'),
 ('DP03_0061PM', 'INC11_PM'),
 ('DP03_0022PE', 'COM05_PC'),
 ('DP02_0003PM', 'HSHD03_PM'),
 ('DP03_0023E', 'COM06_E')]

In [21]:
#Rename the census variables to ri geodatabase variables
acsdata.rename(columns=cv_to_db,inplace=True)
acsdata.head(3)

,GEOID2,GEOLABEL,HSHD01_E,HSHD01_M,HSHD01_PC,HSHD01_PM,HSHD02_E,HSHD02_M,HSHD02_PC,HSHD02_PM,...,INC11_PC,INC11_PM,INC12_E,INC12_M,INC12_PC,INC12_PM,INC13_E,INC13_M,INC13_PC,INC13_PM
GEOID,,,,,,,,,,,,,,,,,,,,,
1400000US44001030100,44001030100,"Census Tract 301, Bristol County, Rhode Island",1828,81,1828,NaN,1258,92,68.8,4.8,...,18.4,4.9,96875,13302,NaN,NaN,121685,12991,NaN,NaN
1400000US44001030200,44001030200,"Census Tract 302, Bristol County, Rhode Island",1207,83,1207,NaN,862,94,71.4,6.9,...,14.7,5.7,106875,11018,NaN,NaN,127146,16401,NaN,NaN
1400000US44001030300,44001030300,"Census Tract 303, Bristol County, Rhode Island",1654,77,1654,NaN,1316,79,79.6,4.6,...,23.6,4.8,131757,8480,NaN,NaN,160177,15896,NaN,NaN


## Write to Database
Update list of variables and data types, build create table string, create datatable in temporary database.


In [22]:
dfexcel.replace({'dtype': {'int': 'INTEGER', 'float': 'REAL'}},inplace=True)
dfexcel.census_label.replace({'!!': ' - '},inplace=True, regex=True)
dfexcel.head()

,db_var,census_var,census_label,dtype
0,HSHD01_E,DP02_0001E,Estimate - HOUSEHOLDS BY TYPE - Total households,INTEGER
1,HSHD01_M,DP02_0001M,Margin of Error - HOUSEHOLDS BY TYPE - Total h...,INTEGER
2,HSHD01_PC,DP02_0001PE,Percent Estimate - HOUSEHOLDS BY TYPE - Total ...,INTEGER
3,HSHD01_PM,DP02_0001PM,Percent Margin of Error - HOUSEHOLDS BY TYPE -...,INTEGER
4,HSHD02_E,DP02_0002E,Estimate - HOUSEHOLDS BY TYPE - Total househol...,INTEGER


In [24]:
vardict=dfexcel.set_index('db_var').T.to_dict('list')
random.sample(list(vardict.items()), 2)

[('HSHD04_PM',
  ['DP02_0015PM',
   'Percent Margin of Error - HOUSEHOLDS BY TYPE - Total households - Average household size',
   'INTEGER']),
 ('INC04_M',
  ['DP03_0054M',
   'Margin of Error - INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999',
   'INTEGER'])]

In [25]:
con = sqlite3.connect(dbname) 
cur = con.cursor()

In [26]:
cur.execute('DROP TABLE IF EXISTS {}'.format(tabname))
dbstring="""
CREATE TABLE {} (
GEOID TEXT,
GEOID2 TEXT NOT NULL PRIMARY KEY,
GEOLABEL TEXT,
""".format(tabname)

for k,v in vardict.items():
    dbstring=dbstring+k+' '+v[2]+', \n'
    
dbstring=dbstring[:-3]
dbstring=dbstring+');'
print(dbstring)


CREATE TABLE tracts_2018acs1 (
GEOID TEXT,
GEOID2 TEXT NOT NULL PRIMARY KEY,
GEOLABEL TEXT,
HSHD01_E INTEGER, 
HSHD01_M INTEGER, 
HSHD01_PC INTEGER, 
HSHD01_PM INTEGER, 
HSHD02_E INTEGER, 
HSHD02_M INTEGER, 
HSHD02_PC REAL, 
HSHD02_PM REAL, 
HSHD03_E INTEGER, 
HSHD03_M INTEGER, 
HSHD03_PC REAL, 
HSHD03_PM REAL, 
HSHD04_E REAL, 
HSHD04_M REAL, 
HSHD04_PC INTEGER, 
HSHD04_PM INTEGER, 
HSHD05_E REAL, 
HSHD05_M REAL, 
HSHD05_PC INTEGER, 
HSHD05_PM INTEGER, 
FERT01_E INTEGER, 
FERT01_M INTEGER, 
FERT01_PC INTEGER, 
FERT01_PM INTEGER, 
SCH01_E INTEGER, 
SCH01_M INTEGER, 
SCH01_PC INTEGER, 
SCH01_PM INTEGER, 
SCH02_E INTEGER, 
SCH02_M INTEGER, 
SCH02_PC REAL, 
SCH02_PM REAL, 
SCH03_E INTEGER, 
SCH03_M INTEGER, 
SCH03_PC REAL, 
SCH03_PM REAL, 
SCH04_E INTEGER, 
SCH04_M INTEGER, 
SCH04_PC REAL, 
SCH04_PM REAL, 
SCH05_E INTEGER, 
SCH05_M INTEGER, 
SCH05_PC REAL, 
SCH05_PM REAL, 
SCH06_E INTEGER, 
SCH06_M INTEGER, 
SCH06_PC REAL, 
SCH06_PM REAL, 
EDU01_E INTEGER, 
EDU01_M INTEGER, 
EDU01_PC INTE

In [27]:
cur.execute(dbstring)

In [28]:
acsdata.to_sql(name=tabname, if_exists='append', index=True, con=con)

In [29]:
cur.execute('SELECT COUNT(*) FROM {};'.format(tabname))
rows = cur.fetchone()
print(rows[0], 'records written to', tabname)

244 records written to tracts_2018acs1


In [30]:
cur.execute('SELECT * FROM {} LIMIT 1;'.format(tabname))
col_names = [cn[0] for cn in cur.description]
print(len(col_names), 'columns written to', tabname)
#Number should be same as number in df acsdata plus 1, since index not included in df count

251 columns written to tracts_2018acs1


In [31]:
con.close()

## Metadata Table
DO NOT RERUN THIS SECTION FOR MULTIPLE GEOGRAPHIES. In the RI Geodatabase there is only one metadata table for all of the ACS tables (acs1 and acs2) for all geographies. For whichever geography is processed first, set action variable to 'create' and run this entire series of blocks for the acs1 table. For the acs2 table, set the action variable to 'append' and skip the table creation and identifier insertion blocks.

In [32]:
#Change table name and specify an action - you're creating the table for the first time with acs1 variables, 
#or appending the tables with acs2 variables

metatab='acslookup2018'
action='create' # 'create' or 'append'

In [33]:
con = sqlite3.connect(dbname) 
cur = con.cursor()

In [34]:
#Only run this block when creating initial table
if action=='create':
    mdstring="""
    CREATE TABLE {} (
    tabnum TEXT,
    est_id TEXT,
    est_value TEXT);
    """.format(metatab)
    cur.execute(mdstring)
else:
    print('Block not executed because "create" not selected as an action in earlier block')

OperationalError: table acslookup2018 already exists

In [ ]:
#Only run this block when creating initial table
if action=='create':
    exstring="""
        INSERT INTO {} VALUES('both','NOTE','Each variable has 4 values that are identified by a particular suffix: E for estimate, M for margin of error for the estimate, PC for percent total, and PM for margin of error for the percent total');
        INSERT INTO {} VALUES('both','GEOID','Id');
        INSERT INTO {} VALUES('both','GEOID2','Id2');
        INSERT INTO {} VALUES('both','GEOLABEL','Geography');
        """.format(metatab,metatab,metatab,metatab)
    cur.executescript(exstring)
    con.commit()
else:
    print('Block not executed because "create" not selected as an action in earlier block')

In [ ]:
#Run when creating table or when appending records
#Keys and values - db ids and labels - are simplified and truncated to 1 entry for each 4-column group (E,M,PC,PM)
if action in ('create','append'):
    for mk, mv in vardict.items():
        if mk.endswith('_E'):
            cur.execute("INSERT INTO {} values(?,?,?)".format(metatab),(worksheet,mk[:-2],mv[1][11:]))
    con.commit()
else:
    print('Block not executed because action not specified in earlier block')

In [ ]:
cur.execute('SELECT COUNT(*) FROM {};'.format(metatab))
rows = cur.fetchone()
print(rows[0], 'records in', metatab)

In [ ]:
action=''
con.close()